# Machine Learning

## Libs

In [1]:
import os
import re

import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
from tqdm.auto import tqdm

# My modules
import parsing
import posing
import plotting
import feature_extraction

## Sampling Data

Here, we sample 1000 random frames from a single animation.

In [2]:
animation_df = pd.read_pickle('../data/subjects_small/02/02_01.pkl')
animation_df

position  \
frame joint                                                          
0     root                            [10.4194, 16.7048, -30.1003]   
      lhipjoint                       [10.4194, 16.7048, -30.1003]   
      lfemur     [11.836805479108595, 14.853861115486746, -29.1...   
      ltibia     [10.753685157798868, 8.359600144520844, -25.34...   
      lfoot      [10.165121459060382, 1.1663749359201203, -24.3...   
...                                                            ...   
342   rradius    [7.946601264912017, 19.53822358683591, 27.5546...   
      rwrist     [7.925319394386008, 16.27373977695335, 26.7414...   
      rhand      [7.914678427498826, 14.641493020379947, 26.334...   
      rfingers   [7.674591600343146, 13.951704256163172, 26.341...   
      rthumb     [7.914678427498826, 14.641493020379947, 26.334...   

                                                          rotation  \
frame joint                                                          
0     root       [[0.983984031928632, 0.059845803761117373, -0....   
      lhipjoint  [[0.983984031928632, 0.059845803761117373, -0....   
      lfemur     [[0.8727218542384726, 0.46943225994158017, -0....   
      ltibia     [[0.8938815275245334, 0.41129653538509015, -0....   
      lfoot      [[0.9121353206749282, 0.4037221898173864, 0.07...   
...                                                            ...   
342   rradius    [[0.0063257757660457135, -0.9365475805825817, ...   
      rwrist     [[0.0063257757660457, -0.9986536893802985, 0.0...   
      rhand      [[0.3287032515978878, -0.9430258850567821, 0.0...   
      rfingers   [[0.20919693697356634, -0.9765143568115262, 0....   
      rthumb     [[-0.06258141496331021, -0.9894346324556459, 0...   

                                                         direction  \
frame joint                                                          
0     root                                         [0.0, 0.0, 0.0]   
      lhipjoint  [0.5609244013869094, -0.7324910204610584, 0.38...   
      lfemur     [-0.14263388005464087, -0.855215826120026, 0.4...   
      ltibia     [-0.08076711518168062, -0.9871081927004208, 0....   
      lfoot      [0.05093377846176101, 0.0835806756632005, 0.99...   
...                                                            ...   
342   rradius    [-0.006325775723999597, -0.9703278859209057, -...   
      rwrist     [-0.00632577572120535, -0.9703278859165606, -0...   
      rhand      [-0.32870325155554475, -0.9443908787944982, 0....   
      rfingers   [-0.20919693692973385, -0.9778703486944745, 0....   
      rthumb     [0.13672522631718434, -0.9600105140601315, 0.2...   

                                         axis    length  \
frame joint                                               
0     root                    [0.0, 0.0, 0.0]         0   
      lhipjoint               [0.0, 0.0, 0.0]   2.52691   
      lfemur                 [0.0, 0.0, 20.0]   7.59371   
      ltibia                 [0.0, 0.0, 20.0]   7.28717   
      lfoot        [-90.0, 7.62852e-16, 20.0]    2.2218   
...                                       ...       ...   
342   rradius             [180.0, 30.0, 90.0]   3.36431   
      rwrist     [-2.22354e-14, -90.0, -90.0]   1.68216   
      rhand      [-4.26681e-14, -90.0, -90.0]  0.730406   
      rfingers   [-8.53362e-14, -90.0, -90.0]  0.588872   
      rthumb     [-90.0, -45.0, -2.85299e-15]  0.845506   

                                      dof  \
frame joint                                 
0     root       [tx, ty, tz, rx, ry, rz]   
      lhipjoint                        []   
      lfemur                 [rx, ry, rz]   
      ltibia                         [rx]   
      lfoot                      [rx, rz]   
...                                   ...   
342   rradius                        [rx]   
      rwrist                         [ry]   
      rhand                      [rx, rz]   
      rfingers                       [rx]   
      rthumb     

In [51]:
num_frames = len(animation_df.index.get_level_values('frame'))
selection = np.random.choice(num_frames, 1000, replace=False)

subset_df = animation_df[animation_df.index.get_level_values('frame').isin(selection)]
subset_df

position  \
frame joint                                                          
9     root                             [10.323, 16.5366, -28.3523]   
      lhipjoint                        [10.323, 16.5366, -28.3523]   
      lfemur     [11.752467282622936, 14.69624305096022, -27.37...   
      ltibia     [10.455958018169085, 8.204593396360726, -23.65...   
      lfoot      [9.856027255446325, 0.942306605721031, -23.700...   
...                                                            ...   
320   rradius    [7.70125065357659, 19.57286047483364, 24.37716...   
      rwrist     [7.552461499933417, 16.22232558194841, 24.6424...   
      rhand      [7.478066701982174, 14.547053155977888, 24.775...   
      rfingers   [7.230010465474555, 13.931440375760554, 25.079...   
      rthumb     [7.478066701982174, 14.547053155977888, 24.775...   

                                                          rotation  \
frame joint                                                          
9     root       [[0.9832524716288616, 0.04993063510427259, -0....   
      lhipjoint  [[0.9832524716288616, 0.04993063510427259, -0....   
      lfemur     [[0.8595795996091795, 0.4945530554124446, -0.1...   
      ltibia     [[0.889816784319976, 0.4114770731916519, -0.19...   
      lfoot      [[0.9099420796777122, 0.4081325687487219, 0.07...   
...                                                            ...   
320   rradius    [[0.04422575618539121, -0.9858366303165926, 0....   
      rwrist     [[0.04422575618539121, -0.9740993739763942, 0....   
      rhand      [[0.33961418243767283, -0.9132747759625074, 0....   
      rfingers   [[0.22371378497545166, -0.9483463247854835, 0....   
      rthumb     [[0.0366057691315529, -0.9643709295706748, 0.2...   

                                                         direction  \
frame joint                                                          
9     root                                         [0.0, 0.0, 0.0]   
      lhipjoint  [0.5656977425483833, -0.7283033226508979, 0.38...   
      lfemur     [-0.17073462964135472, -0.8548719472562811, 0....   
      ltibia     [-0.08232698876556475, -0.9965853398012804, -0...   
      lfoot      [0.052441996531323504, -0.1259417570321705, 0....   
...                                                            ...   
320   rradius    [-0.04422575614113227, -0.9959055178878367, 0....   
      rwrist     [-0.04422575614165337, -0.9959055178880264, 0....   
      rhand      [-0.33961418239666563, -0.8428364227803905, 0....   
      rfingers   [-0.22371378493288352, -0.8861454019941821, 0....   
      rthumb     [0.1593842183325654, -0.8087503156678316, 0.56...   

                                         axis    length  \
frame joint                                               
9     root                    [0.0, 0.0, 0.0]         0   
      lhipjoint               [0.0, 0.0, 0.0]   2.52691   
      lfemur                 [0.0, 0.0, 20.0]   7.59371   
      ltibia                 [0.0, 0.0, 20.0]   7.28717   
      lfoot        [-90.0, 7.62852e-16, 20.0]    2.2218   
...                                       ...       ...   
320   rradius             [180.0, 30.0, 90.0]   3.36431   
      rwrist     [-2.22354e-14, -90.0, -90.0]   1.68216   
      rhand      [-4.26681e-14, -90.0, -90.0]  0.730406   
      rfingers   [-8.53362e-14, -90.0, -90.0]  0.588872   
      rthumb     [-90.0, -45.0, -2.85299e-15]  0.845506   

                                      dof  \
frame joint                                 
9     root       [tx, ty, tz, rx, ry, rz]   
      lhipjoint                        []   
      lfemur                 [rx, ry, rz]   
      ltibia                         [rx]   
      lfoot                      [rx, rz]   
...                                   ...   
320   rradius                        [rx]   
      rwrist                         [ry]   
      rhand                      [rx, rz]   
      rfingers                       [rx]   
      rthumb     

In [88]:
input_joints = ['lhand', 'rhand', 'head']
output_joints = ['root', 'thorax', 'lhumerus', 'rhumerus']

input_df = subset_df.loc[:, input_joints, :][['position']]
input_df = input_df.groupby('frame').agg({'position': np.stack}).rename(columns={'position': 'input'})

output_df = subset_df.loc[:, output_joints, :][['position']]
output_df = output_df.groupby('frame').agg({'position': np.stack}).rename(columns={'position': 'output'})

ml_df = pd.concat([input_df, output_df], axis=1)
display(*ml_df.iloc[0])

array([[ 13.9559676 ,  14.17438644, -29.42725872],
       [  5.89062239,  15.56910523, -23.25388149],
       [  9.92245792,  25.80677787, -28.32365251]])

array([[ 10.323     ,  16.5366    , -28.3523    ],
       [ 10.10991416,  20.64891823, -28.2494241 ],
       [ 13.4868048 ,  23.78372224, -27.81456796],
       [  6.4928386 ,  23.32725924, -28.50934491]])